In [26]:
import math
import operator as op
# https://norvig.com/lispy.html

In [28]:
program = "(begin (define r 10) (* pi (* r r)))"

In [68]:
Symbol = str
Number = int | float
Atom = Symbol | Number
List = list
Exp = Atom | list

In [14]:
def tokenize(program: str) -> list[str]:
    return (
        program
        .replace("(", " ( ")
        .replace(")", " ) ")
        .split()
    )

In [15]:
def atom(token) -> Atom:
    try:
        return int(token)
    except ValueError:
        try:
            return float(token)
        except ValueError:
            return Symbol(token)


def read_from_tokens(tokens: list[str]) -> Exp:
    if len(tokens) == 0:
        raise SyntaxError("Unexpected EOF.")
        
    token = tokens.pop(0)
    if token == "(":
        # read a sub expression
        L = []
        while tokens[0] != ")":  # end of the sub expression
            L.append(read_from_tokens(tokens))  # tokens are mutated

        tokens.pop(0) # pop the ending ")" of the sub expression
        return L
                     
    if token == ")":
        raise SyntaxError("Unexpected ).")
                     
    return atom(token)

In [18]:
def parse(program: str) -> Exp:
    tokens = tokenize(program)
    return read_from_tokens(tokens)

In [19]:
parse(program)

['begin', ['define', 'r', 10], ['*', 'pi', ['*', 'r', 'r']]]

In [20]:
assert parse(program) == ["begin", ["define", "r", 10], ["*", "pi", ["*", "r", "r"]]]

In [115]:
class Env(dict):
    
    def __init__(
        self,
        params: tuple[str] = (),
        args: tuple = (),
        outer: Env | None = None
    ):
        self.update(zip(params, args))
        self.outer = outer
        
    def find(self, var: str):
        # Find the correct environment for `var` 
        return self if var in self else self.outer.find(var)
    
    
class Procedure:
    def __init__(self, params: tuple[str], body: Exp, env: Env):
        self.params = params
        self.body = body
        # global environment if the procedure is at the top level
        self.env = env
        
    def __call__(self, *args) -> Exp:
        # calling the procedure create local environment for the call
        # connected to the outer "global" environment (stored in `self.env`)
        return eval_(self.body,  Env(self.params, args, self.env))

In [116]:
def standard_env() -> Env:
    env = Env()
    env.update(
        {
            "+": op.add,
            "-": op.sub,
            "*": op.mul,
            "/": op.truediv, 
            ">": op.gt,
            "<": op.lt,
            ">=": op.ge,
            "<=": op.le,
            "=": op.eq,
            "abs": abs,
            "exp": math.exp,
            "log": math.log,
            "pi": math.pi,
            "append":  op.add,  
            "apply":   lambda proc, args: proc(*args),
            "begin":   lambda *x: x[-1],  # returns the value of the last expression
            "car":     lambda x: x[0],
            "cdr":     lambda x: x[1:], 
            "cons":    lambda x, y: [x] + y,
            "eq?":     op.is_, 
            "expt":    pow,
            "equal?":  op.eq, 
            "length":  len, 
            "list":    lambda *x: List(x), 
            "list?":   lambda x: isinstance(x, List), 
            "map":     map,
            "max":     max,
            "min":     min,
            "not":     op.not_,
            "null?":   lambda x: x == [], 
            "number?": lambda x: isinstance(x, Number),  
            "print":   print,
            "procedure?": callable,
            "round":   round,
            "symbol?": lambda x: isinstance(x, Symbol),     
        }
    )
    return env
    
    
global_env = standard_env()

In [204]:
def eval_(exp: Exp, env: Env = global_env) -> Exp:
    if isinstance(exp, Symbol):
        return env.find(exp)[exp]  # variable reference
    elif not isinstance(exp, List):
        return exp  # constant
    
    op, *args = exp
    if op == "quote":
        return args[0]  
    elif op == "if":
        # first element is the if keyword
        (test, conseq, alt) = args
        exp_to_eval = conseq if eval_(test, env) else alt
        return eval_(exp_to_eval, env)
    elif op == "define":
        # first element is the define keyword
        (symbol, exp_to_eval) = args
        env[symbol] = eval_(exp_to_eval, env)

    elif op == "set!":
        (symbol, exp_to_eval) = args
        env.find(symbol)[symbol] = eval_(exp_to_eval, env) 
    elif op == "lambda":
        (params, body) = args
        return Procedure(params, body, env)
    else:
        # procedure call
        proc = eval_(op, env)
        proc_args = [eval_(proc_arg, env) for proc_arg in args]
        # if proc is env["begin"]:
        #     print(proc_args)
        #     print(exp)
        return proc(*proc_args)

In [200]:
program = "(begin (define r 10) (* pi (* r r)))"
assert eval_(parse(program)) == 314.1592653589793

[None, 314.1592653589793]
['begin', ['define', 'r', 10], ['*', 'pi', ['*', 'r', 'r']]]


In [201]:
proc_args = [None, 314.1592653589793]

In [202]:
global_env["begin"](*proc_args)

314.1592653589793

In [205]:
program = "(define circle-area (lambda (r) (* pi (* r r))))"
eval_(parse(program))
eval_(parse("(circle-area 3)"))

28.274333882308138

In [207]:
program = "(begin (define circle-area (lambda (r) (* pi (* r r)))) (circle-area 3))"
eval_(parse(program))

28.274333882308138